In [1]:
from __future__ import print_function 
import cv2 
import numpy as np 
MAX_FEATURES = 500 
GOOD_MATCH_PERCENT = 0.15

In [2]:
def alignImages(im1, im2):
    im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
    im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
    orb = cv2.ORB_create(MAX_FEATURES)
    keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, None)
    keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, None)
    matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
    matches = matcher.match(descriptors1, descriptors2, None)
    imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
    cv2.imwrite("matches.jpg", imMatches)
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)
 
    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt

    h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)

    height, width, channels = im2.shape
    im1Reg = cv2.warpPerspective(im1, h, (width, height))
    
    return im1Reg, h

In [3]:
if __name__ == '__main__':
    refFilename = "img.jpg"
    print("Reading Reference Image:", refFilename)
    imReference = cv2.imread(refFilename, cv2.IMREAD_COLOR)
 
    imFilename = "img1.jpg"
    print("Reading Image to align:", imFilename)
    im = cv2.imread(imFilename, cv2.IMREAD_COLOR)

    print("Aligning images ...")
    imReg, h = alignImages(im, imReference)
 
    outFilename = "aligned.jpg"
    print("Saving Aligned Image:", outFilename);
    cv2.imwrite(outFilename, imReg)
    print("Estimated Homography: \n",h)

Reading Reference Image: img.jpg
Reading Image to align: img1.jpg
Aligning images ...
Saving Aligned Image: aligned.jpg
Estimated Homography: 
 [[ 1.72029700e-01 -5.63428510e-01  1.67078983e+02]
 [ 1.59010594e-01 -5.10175442e-01  1.50786031e+02]
 [ 1.04203559e-03 -3.37482154e-03  1.00000000e+00]]
